In [137]:
import json
import os

In [138]:
path = "ExperimentData//"
path = "SampleSessionLayerableOnly//"
path = "1652473795203//"
path = "1652484257598//"
dir_list = os.listdir(path)

# sorted(dir_list, )

print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)


Files and directories in ' 1652484257598// ' :
['RecordStudy_Session_0_1652484257598.json', 'RecordStudy_Session_10_1652484257598.json', 'RecordStudy_Session_11_1652484257598.json', 'RecordStudy_Session_12_1652484257598.json', 'RecordStudy_Session_13_1652484257598.json', 'RecordStudy_Session_14_1652484257598.json', 'RecordStudy_Session_15_1652484257598.json', 'RecordStudy_Session_16_1652484257598.json', 'RecordStudy_Session_17_1652484257598.json', 'RecordStudy_Session_18_1652484257598.json', 'RecordStudy_Session_19_1652484257598.json', 'RecordStudy_Session_1_1652484257598.json', 'RecordStudy_Session_20_1652484257598.json', 'RecordStudy_Session_21_1652484257598.json', 'RecordStudy_Session_22_1652484257598.json', 'RecordStudy_Session_23_1652484257598.json', 'RecordStudy_Session_24_1652484257598.json', 'RecordStudy_Session_25_1652484257598.json', 'RecordStudy_Session_26_1652484257598.json', 'RecordStudy_Session_27_1652484257598.json', 'RecordStudy_Session_28_1652484257598.json', 'RecordSt

In [139]:
appEvents = []
codesEvents = []
experimentEvents = []

def parseData(json_data):
    frames = json_data["frames"]
    for frame in frames:
        appEvent = frame["appEvents"]
        codeEvent = frame["codesEvents"]
        experimentEvent = frame["experimentEvents"]
        if appEvent:
            appEvents.append(appEvent)
        if codeEvent:
            codesEvents.append(codeEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)


In [140]:

for filename in dir_list:
    data = []
    if filename.endswith(".json"):
        # Prints only text file present in My Folder
        print(filename)
        firstFileFlag = False
        if "Session_0" in filename:
            firstFileFlag = True
        with open(path + filename, 'r') as f:
            json_data = json.load(f)
            if(firstFileFlag):
                #data.append(json_data["obj"]["sessionRecording"])
                parseData(json_data["obj"]["sessionRecording"])
            else:
                #data.append(json.load(f))
                parseData(json_data)

RecordStudy_Session_0_1652484257598.json
RecordStudy_Session_10_1652484257598.json
RecordStudy_Session_11_1652484257598.json
RecordStudy_Session_12_1652484257598.json
RecordStudy_Session_13_1652484257598.json
RecordStudy_Session_14_1652484257598.json
RecordStudy_Session_15_1652484257598.json
RecordStudy_Session_16_1652484257598.json
RecordStudy_Session_17_1652484257598.json
RecordStudy_Session_18_1652484257598.json
RecordStudy_Session_19_1652484257598.json
RecordStudy_Session_1_1652484257598.json
RecordStudy_Session_20_1652484257598.json
RecordStudy_Session_21_1652484257598.json
RecordStudy_Session_22_1652484257598.json
RecordStudy_Session_23_1652484257598.json
RecordStudy_Session_24_1652484257598.json
RecordStudy_Session_25_1652484257598.json
RecordStudy_Session_26_1652484257598.json
RecordStudy_Session_27_1652484257598.json
RecordStudy_Session_28_1652484257598.json
RecordStudy_Session_29_1652484257598.json
RecordStudy_Session_2_1652484257598.json
RecordStudy_Session_30_1652484257598.

In [141]:

#experimentEvents[0][0]
experimentEvents
pinCodes = experimentEvents[0][0]["pinCodes"]

for exper in experimentEvents:
    for exp in exper:
        if exp["eventName"] == "EndTask":
            print(exp)


{'unixTime': 1652485371718, 'systemTime': '16-42-51-71', 'userID': '1652484257598', 'sessionType': 'immersive', 'eventName': 'EndTask', 'pinRandomSeed': 1312, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 6}
{'unixTime': 1652486051838, 'systemTime': '16-54-11-83', 'userID': '1652484257598', 'sessionType': 'layerable', 'eventName': 'EndTask', 'pinRandomSeed': 1312, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 8}


In [142]:
import pandas
import itertools
pandas.options.display.max_rows = None

In [143]:
experimentEvents
experimentsUnpacked = list(itertools.chain(*experimentEvents))

experimentsDf = pandas.DataFrame(experimentsUnpacked)
experimentsDf

,unixTime,systemTime,userID,sessionType,eventName,pinRandomSeed,pinNumCodes,pinCodeLength,pinNumPieces,pinCodes,score
0,1652484771710,16-32-51-70,1652484257598,immersive,StartTask,1312,200,6,3,"[{'Value': 233429, 'Label': 'UW', 'Pieces': [{...",0
1,1652485371718,16-42-51-71,1652484257598,immersive,EndTask,1312,200,6,3,[],6
2,1652485451840,16-44-11-83,1652484257598,layerable,StartTask,1312,200,6,3,"[{'Value': 233429, 'Label': 'UW', 'Pieces': [{...",0
3,1652486051838,16-54-11-83,1652484257598,layerable,EndTask,1312,200,6,3,[],8


In [144]:
appEvents
appsUnpacked = list(itertools.chain(*appEvents))

In [151]:
appsDf = pandas.DataFrame(appsUnpacked)
appsDf.sort_values(by='eventTime')

tutorialEvents = appsDf[appsDf['name'] == 'Tutorial']
annotationEvents = appsDf[appsDf['name'] == 'AnnotationsMenu']
codeEntryEvents = appsDf[appsDf['name'] == 'Code Entry']
itemInspectorEvents = appsDf[appsDf['name'] == 'Item Inspector']
deviceGroupsEvents = appsDf[appsDf['name'] == 'Device Groups']
atlasEvents = appsDf[appsDf['name'] == 'Atlas']

atlasStarts = atlasEvents[atlasEvents['activity'] == 'Start']
atlasStops = atlasEvents[atlasEvents['activity'] == 'Stop']

immersiveStartTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'immersive') & (experimentsDf['eventName'] == 'StartTask')]['unixTime'].to_numpy()[0]
immersiveEndTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'immersive') & (experimentsDf['eventName'] == 'EndTask')]['unixTime'].to_numpy()[0]

layerableStartTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'layerable') & (experimentsDf['eventName'] == 'StartTask')]['unixTime'].to_numpy()[0]
layerableEndTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'layerable') & (experimentsDf['eventName'] == 'EndTask')]['unixTime'].to_numpy()[0]

# layerableCodeEntryEvents = codeEntryEvents.loc[(codeEntryEvents['unixTime'] > layerableStartTime) & (codeEntryEvents['unixTime'] < layerableEndTime)]
# layerableCodeEntryEvents
experimentsDf

,unixTime,systemTime,userID,sessionType,eventName,pinRandomSeed,pinNumCodes,pinCodeLength,pinNumPieces,pinCodes,score
0,1652484771710,16-32-51-70,1652484257598,immersive,StartTask,1312,200,6,3,"[{'Value': 233429, 'Label': 'UW', 'Pieces': [{...",0
1,1652485371718,16-42-51-71,1652484257598,immersive,EndTask,1312,200,6,3,[],6
2,1652485451840,16-44-11-83,1652484257598,layerable,StartTask,1312,200,6,3,"[{'Value': 233429, 'Label': 'UW', 'Pieces': [{...",0
3,1652486051838,16-54-11-83,1652484257598,layerable,EndTask,1312,200,6,3,[],8
